In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [2]:
def intrpf(xi, x, y):
    """
    Function to interpolate between data points using Lagrange polynomial (quadratic).
    :param xi: The x value where interpolation is computed
    :param x: Vector of x coordinates of data points (3 values)
    :param y: Vector of y coordinates of data points (3 values)
    :return yi: The interpolation polynomial evaluated at xi
    """
    # Calculate yi = p(xi) using the Lagrange polynomial
    yi = (
        ((xi - x[1]) * (xi - x[2])) / ((x[0] - x[1]) * (x[0] - x[2])) * y[0]
        + ((xi - x[0]) * (xi - x[2])) / ((x[1] - x[0]) * (x[1] - x[2])) * y[1]
        + ((xi - x[0]) * (xi - x[1])) / ((x[2] - x[0]) * (x[2] - x[1])) * y[2]
    )

    return yi

In [35]:
def routine(tau, method):

    # user-varied parameters: (to be put in eval/input cases)
#     method = 2 # 0 for Euler, 1 for Euler-Cromer, 2 for midpoint
    y0 = 0 # meters
    speed = 50 # m/s
    theta = 45 # degrees
    timeLimit = 10 # seconds

    methDict = ['Euler', 'Euler-Cromer', 'Midpoint']
    r0 = np.array([0., y0])  # Initial vector position
    v0 = np.array([speed * np.cos(theta*np.pi/180), speed * np.sin(theta*np.pi/180)])  # initial velocity
    r = np.copy(r0)  # Set initial position
    v = np.copy(v0)  # Set initial velocity

    # Set physical parameters (mass, Cd, etc.)
    Cd = 0.35  # Drag coefficient (dimensionless)
    area = 4.3e-3  # Cross-sectional area of projectile (m^2)
    mass = 0.145   # Mass of projectile (kg)
    grav = 9.81    # Gravitational acceleration (m/s^2)

    # Set air resistance flag
    # airFlag = eval(input('Add air resistance? (Yes: 1 No: 0)'))
    airFlag = 0
    if airFlag == 0:
        rho = 0.       # No air resistance
        air_text = '(no air)'
    else:
        rho = 1.2     # Density of air (kg/m^3)
        air_text = '(with air)'
    air_const = -0.5*Cd*rho*area/mass   # Air resistance constant

    # * Loop until ball hits ground or max steps completed
    # tau = eval(input('Enter timestep dt in seconds: '))  # (sec)
    # tau = .1
    maxstep = 1000
    laststep = maxstep

    # Set up arrays for data
    xplot = np.empty(maxstep)
    yplot = np.empty(maxstep)

    x_noAir = np.empty(maxstep)
    y_noAir = np.empty(maxstep)

    for istep in range(maxstep):
        t = istep * tau  # Current time

        # Record computed position for plotting
        xplot[istep] = r[0]
        yplot[istep] = r[1]

        x_noAir[istep] = r0[0] + v0[0]*t
        y_noAir[istep] = r0[1] + v0[1]*t - 0.5*grav*t**2

        # Calculate the acceleration of the ball
        accel = air_const * np.linalg.norm(v) * v  # Air resistance
        accel[1] = accel[1] - grav # update y acceleration to include gravity

        if method == 0:
            # Calculate the new position and velocity using Euler's method.
            r = r + tau * v  # Euler step
            v = v + tau * accel
        elif method == 1:
            # calculate the new position and velocity using Euler-Cromer method
            # note how the reassignment of v is required to happen before r as 
            # the new v is used in the new r
            v = v + tau * accel
            r = r + tau * v
        elif method == 2:
            # Calculate the new position and velocity using the midpoint method
            r = r + tau * v + 0.5*accel*(tau**2)
            v = v + tau * accel

    #     If the ball reaches the ground (i.e. y < 0) or max runtime reached, break out of the loop
        if r[1] <= 0 or (istep * tau >= timeLimit):
            laststep = istep + 1
            xplot[laststep] = r[0]  # Record last values completed
            yplot[laststep] = r[1]
            break

    # # # Print maximum range and time of flight
    # print('Maximum range is {0:.2f} meters'.format(r[0]))
    # print('Time of flight is {0:.1f} seconds'.format(laststep * tau))

    xinter = xplot[laststep-2:laststep+1] 
    yinter = yplot[laststep-2:laststep+1]
    xguess = (xinter[-1] + xinter[-2])/2 # bisect method
    tol = 1e-3

    # find where interpolated function reaches zero within chosen tolerance
    xguess_arr = np.arange(xguess, xinter[-1], tol)
    for i,x in enumerate(xguess_arr):
        if(intrpf(x, xinter, yinter)<0):
            stopiter = i
            break

    max_range = xguess_arr[stopiter]

    # # Print maximum range and time of flight
    # print(f'\nCorrected Maximum range is {max_range:.2f} meters')
    # print(f'Corrected Time of flight is {(laststep*tau):.1f} seconds')

    # # Graph the trajectory of the baseball
    # fig, ax = plt.subplots()
    # ax.set_title('Projectile Motion: ' + air_text)
    # ax.plot(x_noAir[:laststep], y_noAir[:laststep], '-', c='C2', label='Theory (no air)')
    # ax.plot(xplot[:laststep], yplot[:laststep], '+', label=f'{methDict[method]} method')
    # # Mark the location of the ground by a straight line
    # ax.plot(np.array([0.0, x_noAir[laststep-1]]), np.array([0.0, 0.0]), '-', color='k')
    # ax.legend(frameon=False)
    # ax.set_xlabel('Range (m)')
    # ax.set_ylabel('Height (m)')
    # plt.show()
    percentDiff = np.abs(max_range-r[0])/(max_range+r[0])*(.5)
    
    return(percentDiff*100)

In [41]:
# Euler
err = 0
tau = .1
while err <= 1:
    err = routine(tau, 0)
    tau += .01

print(f'tau: {tau} ')


tau: 0.7000000000000005 


In [42]:
# Euler-Cromer
err = 0
tau = .1
while err <= 1:
    err = routine(tau, 1)
    tau += .01
    
print(f'tau: {tau} ')


tau: 0.5900000000000004 


In [43]:
# Midpoint
err = 0
tau = .1
while err <= 1:
    err = routine(tau, 2)
    tau += .01
    
print(f'tau: {tau} ')


tau: 0.6400000000000005 
